In [1]:
import os
from pathlib import Path
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine, text

# Wenn dein Notebook in notebooks/ liegt und .env im Projekt-Root:
load_dotenv(Path("..") / ".env")

DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT", "5432")
DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")          # <- dein Key
DB_PASS = os.getenv("DB_PASSWORD")

assert DB_HOST and DB_NAME and DB_USER, "Fehlende DB-Variablen (DB_HOST/DB_NAME/DB_USER)"

engine = create_engine(
    f"postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
)

# Test
with engine.connect() as conn:
    print(conn.execute(text("SELECT current_user, current_database()")).fetchone())


('postgres', 'mimic')


In [2]:
import pandas as pd
from sqlalchemy import text

AKI_ONLY = True  # False = alle Patienten/Aufenthalte

def q(sql: str) -> pd.DataFrame:
    with engine.connect() as conn:
        return pd.read_sql(text(sql), conn)


In [3]:
# 1) Welche Creatinine-Labitems gibt es?
sql_creat_items = """
SELECT itemid, label
FROM d_labitems
WHERE LOWER(label) LIKE '%creatinine%'
ORDER BY itemid;
"""
df_creat_items = q(sql_creat_items)
df_creat_items.head(30)


,itemid,label
0,50841,"Creatinine, Ascites"
1,50912,Creatinine
2,51021,"Creatinine, Joint Fluid"
3,51032,"Creatinine, Body Fluid"
4,51052,"Creatinine, Pleural"
5,51067,24 hr Creatinine
6,51070,"Albumin/Creatinine, Urine"
7,51073,"Amylase/Creatinine Ratio, Urine"
8,51080,Creatinine Clearance
9,51081,"Creatinine, Serum"


In [4]:
sql_uo_items = """
SELECT itemid, label
FROM d_items
WHERE LOWER(label) LIKE '%urine%'
ORDER BY itemid;
"""
df_uo_items = q(sql_uo_items)
df_uo_items.head(50)


,itemid,label
0,706,Urine [Color]
1,707,Urine Source
2,941,urine culture
3,1011,urine osmolarity
4,1352,urine pH
5,1495,urine ph
6,1880,Urine pH
7,1917,Urine Na+
8,1918,Urine Osm
9,1919,Urine creat
